In [2]:
import os
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime

In [3]:
Registros = []
with open("C:/Analytics/Lista_de_Ruc.csv", mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        Registros.append(row[0])  # Suponiendo que el dato está en la primera columna
Registros

['1790300404001',
 '1391738986001',
 '1391880081001',
 "0891742622001'",
 '1391932105001',
 '1391797486001',
 '1391898800001',
 '1390012949001',
 '1391877986001',
 '1391932601001',
 '1391794495001',
 '1391932573001',
 '1391800673001',
 "0990604282001'",
 '1391822189001',
 '1391815123001',
 '1391933738001']

In [5]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get("https://srienlinea.sri.gob.ec/sri-en-linea/SriPagosWeb/ConsultaDeudasFirmesImpugnadas/Consultas/consultaDeudasFirmesImpugnadas")
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
for registro in Registros:  

    time.sleep(3)

    input_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[1]/div[6]/div[2]/div/div[1]/div/div/div/sri-campo-ruc-cedula/form/div/div[2]/input")))
    # Ingresar el número en el campo de texto
    input_element.send_keys(registro)
    #webdriver.ActionChains(driver).send_keys(Keys.TAB).perform()

    time.sleep(5)
    elemento_xpath = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[1]/div[6]/div[2]/div/div[2]/div[3]/button/span")))
    elemento_xpath.click()
    time.sleep(5)
    #reconocimiento de captcha
    try:
        captcha_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.NAME, "c-suy2n4ul97ka"))
        )
        print("CAPTCHA detectado. Por favor, resuélvalo manualmente...")
        
        # Esperar asíncronamente hasta que desaparezca el CAPTCHA
        while True:
            time.sleep(5)  # Revisar cada 5 segundos
            try:
                driver.find_element(By.NAME, "captcha")
                print("Esperando a que el usuario resuelva el CAPTCHA...")
            except:
                print("CAPTCHA resuelto, continuando...")
                break  # Salir del bucle cuando el CAPTCHA ya no está
    except:
        print("No se detectó CAPTCHA, continuando...")


    try:
        time.sleep(5)
        captcha_presente = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.NAME, "c-suy2n4ul97ka")), timeout = 50)
        if captcha_presente:
            print("CAPTCHA detectado. Resuélvalo manualmente y luego presiona Enter...")
            input("Presiona Enter para continuar después de resolver el CAPTCHA...")
    except:
        time.sleep(5)
        print("No se detectó CAPTCHA, continuando...")

    #Captura ruc
    ruc_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[4]/div[1]/div/div[1]/div/sri-mostrar-informacion-deudor/div[1]/div[1]/div[2]/div[1]/span")))
    ruc = ruc_element.get_attribute("innerText").strip()

    #Captura razón social
    razon_social_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[4]/div[1]/div/div[1]/div/sri-mostrar-informacion-deudor/div[5]/div/div[2]/div[1]")))
    razon_social = razon_social_element.get_attribute("innerText").strip()


    #Captura texto de Deuda
    try:
        adeuda_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[4]/div[1]/div/div[2]/div[4]/span")))
        adeud = adeuda_element.get_attribute("innerText").strip()
    except Exception as e:
        adeud = "Mantiene Deuda"
    
    print(razon_social)

    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(5)

    #Impresión/salida en archivo excel
    fecha_consulta = datetime.now().strftime("%Y-%m-%d")
    hora_consulta = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    #Crear un DataFrame solo con el nuevo dato
    nuevo_dato = pd.DataFrame([{
        "Fecha de Consulta": fecha_consulta,
        "Hora de Consulta": hora_consulta,
        "RUC": ruc,
        "Razon Social": razon_social,
        "Adeuda": adeud
    }])

    nuevo_dato["RUC"] = nuevo_dato["RUC"].astype(str)
       #archivo_salida = os.path.join(os.path.expanduser("~"), "Desktop", "cumplimiento_supercias.xlsx")
    archivo_salida = "C:/Analytics/ADEUDA/adeuda_sri.xlsx"
        #Verificar si la carpeta existe, si no, crearla
    if not os.path.exists("C:/Analytics/ADEUDA"):
        os.makedirs("C:/Analytics/ADEUDA", exist_ok=True)
    if os.path.exists(archivo_salida):
        df_existente = pd.read_excel(archivo_salida, dtype={"RUC": str})
        df_resultado = pd.concat([df_existente, nuevo_dato]).drop_duplicates()
        with pd.ExcelWriter(archivo_salida, mode="w", engine="openpyxl") as writer:
            df_resultado.to_excel(writer, index=False)
    else:
        nuevo_dato.to_excel(archivo_salida, index=False)      
    print("Datos guardados en:", archivo_salida)
    elemento_xpath = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[5]/div/div/button/span[1]")))
    time.sleep(1)
    elemento_xpath.click()

driver.quit()

No se detectó CAPTCHA, continuando...
No se detectó CAPTCHA, continuando...
EXTRACTORA AGRICOLA RIO MANSO EXA S.A.
Datos guardados en: C:/Analytics/ADEUDA/adeuda_sri.xlsx
No se detectó CAPTCHA, continuando...
No se detectó CAPTCHA, continuando...
ENERGY & PALMA ENERGYPALMA S.A.
Datos guardados en: C:/Analytics/ADEUDA/adeuda_sri.xlsx
No se detectó CAPTCHA, continuando...
No se detectó CAPTCHA, continuando...
FABRIPALMA S.A
Datos guardados en: C:/Analytics/ADEUDA/adeuda_sri.xlsx
No se detectó CAPTCHA, continuando...
No se detectó CAPTCHA, continuando...
EXTRACTORA NATURAL ECUADOR S.A. EXTRANATU
Datos guardados en: C:/Analytics/ADEUDA/adeuda_sri.xlsx
No se detectó CAPTCHA, continuando...
No se detectó CAPTCHA, continuando...


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF62AD24C25+3179557]
	(No symbol) [0x00007FF62A9888A0]
	(No symbol) [0x00007FF62A8191CA]
	(No symbol) [0x00007FF62A86FA67]
	(No symbol) [0x00007FF62A86FC9C]
	(No symbol) [0x00007FF62A8C3627]
	(No symbol) [0x00007FF62A897C6F]
	(No symbol) [0x00007FF62A8C02F3]
	(No symbol) [0x00007FF62A897A03]
	(No symbol) [0x00007FF62A8606D0]
	(No symbol) [0x00007FF62A861983]
	GetHandleVerifier [0x00007FF62AD867CD+3579853]
	GetHandleVerifier [0x00007FF62AD9D1D2+3672530]
	GetHandleVerifier [0x00007FF62AD92153+3627347]
	GetHandleVerifier [0x00007FF62AAF092A+868650]
	(No symbol) [0x00007FF62A992FFF]
	(No symbol) [0x00007FF62A98F4A4]
	(No symbol) [0x00007FF62A98F646]
	(No symbol) [0x00007FF62A97EAA9]
	BaseThreadInitThunk [0x00007FFA99B7E8D7+23]
	RtlUserThreadStart [0x00007FFA9B0FBF6C+44]
